# L1C Formatting Eploration

In [1]:
import pandas as pd
import h5py
# from pyhdf.SD import SD, SDC
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
os.chdir('/project/sdscap-shakeri/nasa/UVA_NASA_2021')
import icare
import pickle
import copy
from netCDF4 import Dataset
import netCDF4
os.getcwd()

'/gpfs/gpfs0/project/sdscap-shakeri/nasa/UVA_NASA_2021'

In [2]:
# os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
# session = icare.ICARESession('/project/sdscap-shakeri/nasa/HDF')

In [3]:
# path='PARASOL/L1_B-HDF.v1.00/2008/2008_06_01'
# file_list = list(session.listdir(path))
# file_path = path + '/' + file_list[0]
# session.get_file(file_path)

In [4]:
os.chdir('/project/sdscap-shakeri/nasa/HDF')

In [5]:
f = h5py.File("PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5", "r")

# Create example file from h5

In [6]:
measurement_dict = {}
for cat in ['I_np','I_p','Q','U']:
    print(cat)
    
    if cat == 'I_np':
        tag = cat.replace('_np','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field)])
    else:
        tag = cat.replace('_p','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field) and ('NP' not in field)])

    print("tag:", tag)
    fields.sort()
    print(fields)
    arrays = []
    scales = []
    long_names = []
    fills = []
    units = []
    for field in fields:
            
        print(field, end=", ")
        
        scales.append(f['Data_Directional_Fields'][field].attrs['scale_factor'])
        long_names.append(f['Data_Directional_Fields'][field].attrs['long_name'])
        fills.append(f['Data_Directional_Fields'][field].attrs['_FillValue'])
        units.append(f['Data_Directional_Fields'][field].attrs['units'])

        arrays.append(np.array(f['Data_Directional_Fields'][field]))
        
    if len(np.unique(scales)) == 1:
        scales = scales[0]
    if len(np.unique(fills)) == 1:
        fills = fills[0]
    if len(np.unique(units)) == 1:
        units = units[0]
    
    measurement_dict[cat] = {}
    measurement_dict[cat]['fields'] = fields
    measurement_dict[cat]['scale'] = scales
    measurement_dict[cat]['long_name'] = long_names
    measurement_dict[cat]['fill'] = fills
    measurement_dict[cat]['units'] = units
    measurement_dict[cat]['data'] = np.stack(arrays,axis=3)

I_np
tag: I
['I1020NP', 'I443NP', 'I490P', 'I565NP', 'I670P', 'I763NP', 'I765NP', 'I865P', 'I910NP']
I1020NP, I443NP, I490P, I565NP, I670P, I763NP, I765NP, I865P, I910NP, I_p
tag: I
['I490P', 'I670P', 'I865P']
I490P, I670P, I865P, Q
tag: Q
['Q490P', 'Q670P', 'Q865P']
Q490P, Q670P, Q865P, U
tag: U
['U490P', 'U670P', 'U865P']
U490P, U670P, U865P, 

# Create Final Dictionary and Populate

In [7]:
final_dict = {}
final_dict['observation_data'] = {}
final_dict['observation_data']['I_PARASOL'] = copy.deepcopy(measurement_dict['I_np'])

## Wavelengths

In [8]:
for i in [True, False]:
    if i:
        print("polarized")
        tag = 'I_p'
        field_name = 'polarization_wavelengths'
        lambdas = [int(field.replace('I','').replace('P','')) for field in measurement_dict[tag]['fields']]

    else:
        print("non-polarized")
        tag = 'I_np'
        field_name = 'intensity_wavelengths'
        lambdas = [int(field.replace('I','').replace('NP','').replace('P','')) for field in measurement_dict[tag]['fields']]

    shape = measurement_dict[tag]['data'].shape
    new_shape = []

    lambda_arrs = []

    for lam in lambdas:    
        new_shape = []
        for dim in shape[:-1]:
            new_shape.append(dim)
        # new_shape.append(1)

        lambda_arrs.append(np.full(new_shape, fill_value = np.full(16,lam)))
        # break

    full_lambdas_arr = np.stack(lambda_arrs,axis=3)
    print(full_lambdas_arr.shape)
    
    final_dict['sensor_views_bands'] = {}
    final_dict['sensor_views_bands'][field_name] = {}
    final_dict['sensor_views_bands'][field_name]['scale'] = 1
    final_dict['sensor_views_bands'][field_name]['long_name'] = 'field_name'
    final_dict['sensor_views_bands'][field_name]['fill'] = 32767
    final_dict['sensor_views_bands'][field_name]['units'] = 'tbd'
    final_dict['sensor_views_bands'][field_name]['data'] = full_lambdas_arr

polarized
(3240, 6480, 16, 3)
non-polarized
(3240, 6480, 16, 9)


In [9]:
measurement_dict.keys()

dict_keys(['I_np', 'I_p', 'Q', 'U'])

## DOLP
* Calculate DOLP matrix
    * DOLP = sqrt(Q^2 + U^2)/I

In [10]:
reverse_scale = 1000
new_scale = 1/reverse_scale

In [11]:
scale = measurement_dict['Q']['scale']
scale
fill = measurement_dict['Q']['fill']
fill

32767

In [12]:
# scale, abs
for key in measurement_dict.keys():
    if key == 'I_p':
        I_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        I_arr[np.abs(I_arr) == fill] = 1
    
    elif key == 'Q':
        Q_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        Q_arr[np.abs(Q_arr) == fill] = 1
        
    elif key == 'U':
        U_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        U_arr[np.abs(U_arr) == fill] = 1
        
    else:
        continue

In [13]:
DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)*reverse_scale

<ipython-input-13-bca893dafdb9>:1: RuntimeWarning: divide by zero encountered in true_divide
  DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)*reverse_scale


In [14]:
# rename - DOLP
DOLP_arr_unfltrd[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

In [15]:
indx = np.where((np.abs(I_arr) != (fill * scale)) & (np.abs(Q_arr) != (fill * scale)) & (np.abs(U_arr) != (fill * scale)))

In [18]:
# add to dictionary

final_dict['observation_data']['DOLP_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['DOLP_PARASOL']['scale'] = new_scale
final_dict['observation_data']['DOLP_PARASOL']['long_name'] = 'Degree of linear polarization'
final_dict['observation_data']['DOLP_PARASOL']['fill'] = fill
final_dict['observation_data']['DOLP_PARASOL']['units'] = 'None'
final_dict['observation_data']['DOLP_PARASOL']['data'] = np.round(DOLP_arr_unfltrd).astype(int)

## Q over I, U over I

In [19]:
Q_over_I = np.divide(Q_arr, I_arr)*reverse_scale
U_over_I = np.divide(U_arr, I_arr)*reverse_scale

Q_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

U_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

final_dict['observation_data']['Q_over_I_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['Q_over_I_PARASOL']['scale'] = new_scale
final_dict['observation_data']['Q_over_I_PARASOL']['long_name'] = 'Q over I'
final_dict['observation_data']['Q_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['Q_over_I_PARASOL']['units'] = 'None'
final_dict['observation_data']['Q_over_I_PARASOL']['data'] = np.round(Q_over_I).astype(int)

final_dict['observation_data']['U_over_I_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['U_over_I_PARASOL']['scale'] = new_scale
final_dict['observation_data']['U_over_I_PARASOL']['long_name'] = 'U over I'
final_dict['observation_data']['U_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['U_over_I_PARASOL']['units'] = 'None'
final_dict['observation_data']['U_over_I_PARASOL']['data'] = np.round(U_over_I).astype(int)

<ipython-input-19-6b485eb901d3>:1: RuntimeWarning: divide by zero encountered in true_divide
  Q_over_I = np.divide(Q_arr, I_arr)*reverse_scale
<ipython-input-19-6b485eb901d3>:1: RuntimeWarning: invalid value encountered in true_divide
  Q_over_I = np.divide(Q_arr, I_arr)*reverse_scale
<ipython-input-19-6b485eb901d3>:2: RuntimeWarning: divide by zero encountered in true_divide
  U_over_I = np.divide(U_arr, I_arr)*reverse_scale
<ipython-input-19-6b485eb901d3>:2: RuntimeWarning: invalid value encountered in true_divide
  U_over_I = np.divide(U_arr, I_arr)*reverse_scale


# Angles, Altitude, & Location

In [20]:
final_dict['geolocation_data'] = {}

In [21]:
f['Geolocation_Fields'].keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'column_number', 'land_sea_flag', 'row_number', 'surface_altitude']>

In [22]:
for field in ['thetas','thetav','phi']:
    if field == 'thetas':
        tag = 'solar_zenith'
    elif field == 'thetav':
        tag = 'sensor_zenith'
    elif field == 'phi':
        tag = 'relative_azimuth'
        
    final_dict['geolocation_data'][tag] = {}
    final_dict['geolocation_data'][tag]['scale'] = f['Data_Directional_Fields'][field].attrs['scale_factor']
    final_dict['geolocation_data'][tag]['long_name'] = f['Data_Directional_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['fill'] = f['Data_Directional_Fields'][field].attrs['_FillValue']
    final_dict['geolocation_data'][tag]['units'] = f['Data_Directional_Fields'][field].attrs['units']
    final_dict['geolocation_data'][tag]['data'] = np.array(f['Data_Directional_Fields'][field])

In [23]:
for field in ['Latitude','Longitude','surface_altitude']:
    if field == 'surface_altitude':
        tag = 'altitude'
    else:
        tag = field
        
        
    final_dict['geolocation_data'][tag] = {}

    final_dict['geolocation_data'][tag]['long_name'] = f['Geolocation_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['units'] = f['Geolocation_Fields'][field].attrs['units']
    
    if tag in ['Latitude','Longitude']:
        temp_arr = np.array(f['Geolocation_Fields'][field])
        fill = f['Geolocation_Fields'][field].attrs['_FillValue']
        fill_inds = np.where(temp_arr == fill)
        temp_arr = temp_arr*reverse_scale
        temp_arr[fill_inds] = fill
        fill = int(fill)
        
        final_dict['geolocation_data'][tag]['fill'] = fill
        final_dict['geolocation_data'][tag]['scale'] = new_scale
        final_dict['geolocation_data'][tag]['data'] = np.round(temp_arr).astype(int)
    else:
        final_dict['geolocation_data'][tag]['scale'] = f['Geolocation_Fields'][field].attrs['scale_factor']
        final_dict['geolocation_data'][tag]['data'] = np.array(f['Geolocation_Fields'][field])
        final_dict['geolocation_data'][tag]['fill'] = f['Geolocation_Fields'][field].attrs['_FillValue']


In [24]:
for cat in final_dict.keys():
    for variable in final_dict[cat]:
        print(cat, variable, (final_dict[cat][variable]['data']).dtype)

observation_data I_PARASOL int16
observation_data DOLP_PARASOL int64
observation_data Q_over_I_PARASOL int64
observation_data U_over_I_PARASOL int64
sensor_views_bands intensity_wavelengths int64
geolocation_data solar_zenith uint16
geolocation_data sensor_zenith uint16
geolocation_data relative_azimuth uint16
geolocation_data Latitude int64
geolocation_data Longitude int64
geolocation_data altitude int16


In [25]:


def write_nc(variable_dict, filename):
    
    nc = Dataset(filename, mode='w', format='NETCDF4')

    for cat in variable_dict.keys():
        print("Starting:", cat)
        # Create the category group to store the variables
        nc.createGroup(cat)

        for var in variable_dict[cat].keys():
            print(var)

            # Fill the dimension with variables
            dimensions = []
            for i in range(len(variable_dict[cat][var]['data'].shape)):  
                dim_name = f'{var}_{i}'
                nc.createDimension(dim_name, size=None)
                dimensions.append(dim_name)

            # Create the variable instance
            print('creating variable')
            nc[cat].createVariable(var, datatype='i8', dimensions=dimensions, fill_value=variable_dict[cat][var]['fill'])

            # Create variable metadata
            print('creating the metadataverse')
            nc[cat][var].long_name = variable_dict[cat][var]['long_name']
            nc[cat][var].units = variable_dict[cat][var]['units']
            nc[cat][var].scale_factor = variable_dict[cat][var]['scale']

            # Create variable array 
            print('creating variable array')
            nc[cat][var][:] = variable_dict[cat][var]['data']
            
            print("")
    return nc

In [26]:
os.chdir('/gpfs/gpfs0/project/sdscap-shakeri/nasa/UVA_NASA_2021')

In [27]:
write_nc(final_dict, 'example.nc')

opening new file
I_PARASOL
creating variable
creating the metadataverse
create variable array
DOLP_PARASOL
creating variable
creating the metadataverse
create variable array
Q_over_I_PARASOL
creating variable
creating the metadataverse
create variable array
U_over_I_PARASOL
creating variable
creating the metadataverse
create variable array
intensity_wavelengths
creating variable
creating the metadataverse
create variable array
solar_zenith
creating variable
creating the metadataverse
create variable array
sensor_zenith
creating variable
creating the metadataverse
create variable array
relative_azimuth
creating variable
creating the metadataverse
create variable array
Latitude
creating variable
creating the metadataverse
create variable array
Longitude
creating variable
creating the metadataverse
create variable array
altitude
creating variable
creating the metadataverse
create variable array


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): I_PARASOL_0(3240), I_PARASOL_1(6480), I_PARASOL_2(16), I_PARASOL_3(9), DOLP_PARASOL_0(3240), DOLP_PARASOL_1(6480), DOLP_PARASOL_2(16), DOLP_PARASOL_3(3), Q_over_I_PARASOL_0(3240), Q_over_I_PARASOL_1(6480), Q_over_I_PARASOL_2(16), Q_over_I_PARASOL_3(3), U_over_I_PARASOL_0(3240), U_over_I_PARASOL_1(6480), U_over_I_PARASOL_2(16), U_over_I_PARASOL_3(3), intensity_wavelengths_0(3240), intensity_wavelengths_1(6480), intensity_wavelengths_2(16), intensity_wavelengths_3(9), solar_zenith_0(3240), solar_zenith_1(6480), solar_zenith_2(16), sensor_zenith_0(3240), sensor_zenith_1(6480), sensor_zenith_2(16), relative_azimuth_0(3240), relative_azimuth_1(6480), relative_azimuth_2(16), Latitude_0(3240), Latitude_1(6480), Longitude_0(3240), Longitude_1(6480), altitude_0(3240), altitude_1(6480)
    variables(dimensions): 
    groups: observation_data, sensor_views_bands, geolocation_data